In [2]:
# Vector Store and Retriever abstractions are designed to support retrieval of data
# from vector databases and other sources for integration with LLM workflows
# Concepts are : Documents, Vector stores, and Retrievers

# Setup environment variables
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [3]:
# Document Abstraction: is intended to represent a unit of text and associated metadata
# It's attributes are: page_content (string representing the content) and metadata
# An individual document often represents a chunk of larger document
from langchain_core.documents import Document

# Five documents containing metadata indicating 3 distinct sources
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyality and friendliness",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [11]:
# Vector Stores: common way to store and search over unstructured data.
# Idea is to store numeric vectors that are associated with the text
# As I have LLAMA2 model already installed locally, 
# I am using OllamaEmbedding instead of Openai embedding as it requires subscription
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings

# This will add the documents as embeddings to the vector store
vectorstore = FAISS.from_documents(
    documents,
    embedding=OllamaEmbeddings()
)

In [12]:
# Search the vector store
vectorstore.similarity_search("cat")

[Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'}),
 Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyality and friendliness', metadata={'source': 'mammal-pets-doc'})]

In [13]:
# Search the vector store and get scores
vectorstore.similarity_search_with_score("cat")

[(Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
  20772.293),
 (Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'}),
  20803.564),
 (Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'}),
  20822.049),
 (Document(page_content='Dogs are great companions, known for their loyality and friendliness', metadata={'source': 'mammal-pets-doc'}),
  28548.764)]

In [14]:
# Return documents based on similarity to a embedded query
embedding = OllamaEmbeddings().embed_query("cat")
vectorstore.similarity_search_by_vector(embedding)

[Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'}),
 Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyality and friendliness', metadata={'source': 'mammal-pets-doc'})]

In [15]:
# VectorStore are not Runnable, so it cannot immediately be integrated into LangChain Expression Language.
# LangChain Retrievers are runnables, so they implement a standard set of methods
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1) # select top result

retriever.batch(["rabbit", "dog"])

[[Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'})]]

In [16]:
# Generate a VectorStoreRetriever, providing the type of search we want and the number of answers we would like the model to return i.e. k
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "shark"])

[[Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'})]]

In [17]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

In [19]:
# Creating a Runnable RAG to get answers for our question based on the given context to the LLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

# Passing message to model deciding a structure
prompt = ChatPromptTemplate.from_messages([("human", message)])

# Create a chain initially providing the context and allowing the user input to be the question using RunnablePassThrough, and followed by prompt and llm
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [20]:
# Ask the question as user input and get response based on the given context by the model
response = rag_chain.invoke("tell me about dogs")
response

"Sure! Here's what I know about dogs based on the provided context:\n\nDogs are great companions and are known for their loyalty and friendliness. They are mammals, specifically members of the family Canidae, and are widely kept as pets. Dogs have a strong instinct to protect and care for their human families, and they are often trained for various tasks such as hunting, herding, and assisting people with disabilities. They are also known for their ability to communicate with humans through body language and vocalizations, and they are highly social animals that thrive on interaction and attention from their human companions. Overall, dogs are beloved pets and play an important role in many people's lives."

In [22]:
# Ask the question as user input and get response based on the given context by the model
response = rag_chain.invoke("tell me about Cats")
response

"Sure! Here's what I know about cats based on the provided context:\n\nCats are not mentioned in the given document, so there is no information available about them."

### The key difference between Vector Stores and Retrievers are:-
### Vector Store creates a database like object and we can search for something in it using mechanics like similarity search
### Retrievers on the other hand goes one step ahead of Vector Stores and we can use them inside an LCEL chain making it a smoother process
### By using Retrievers, we abstract the complexity of directly interacting with vector stores, making it easier to integrate into LangChain workflows.